<a href="https://colab.research.google.com/github/pillalamarri-anil/AIML/blob/main/Function_Calling_in_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
end_point = userdata.get('API_END_POINT')

if not api_key:
    raise ValueError( "OPENAI_API_KEY not found! ")

print("API key loaded successfully.")

API key loaded successfully.


In [4]:
import os
from openai import OpenAI

client = OpenAI(api_key=api_key,
            base_url=end_point)
print("OpenAI client ready.")

OpenAI client ready.


In [5]:

response = client.responses.create(
    model="gpt-4o-mini",  # replace with your model deployment name
    input="how is weather today in hyderabad"
)

print("Model says:", response.output_text)

Model says: I can't check real-time weather updates, but you can easily find today's weather in Hyderabad by checking a weather website or app. They provide current temperatures, humidity, and forecasts. How can I assist you further?


In [5]:

response = client.responses.create(
    model="gpt-4o-mini",  # replace with your model deployment name
    input = "What is 1247 * 83 + 19 / 3.7? Give me answer in one line (max 10 words), the exact number, upto 2 decimal places."
)

print("Model says:", response.output_text)

import math
actual = 1247 * 83 + 19 / 3.7
print(f"Actual answer: {actual}")

Model says: 104,066.14
Actual answer: 103506.13513513513


In [6]:
response = client.responses.create(
    model="gpt-4o-mini",  # replace with your model deployment name
    input="Can you give appoximate longitude and latitude of Hyderabad"
)

print("Model says:", response.output_text)

Model says: Hyderabad, India, is approximately located at a latitude of 17.3851° N and a longitude of 78.4867° E.


# Tool calling

In [23]:
import json # Added this line
import requests # Added this line, as requests was used but not imported
from google.colab import userdata

# Define a simple "add" tool — we'll explain the structure in detail next
  # Define the function for the model
  # Ask the model to add — but give it the tool
weather_tool = {
            "type": "function",
            "name": "get_weather_at",
            "description": "Get the weather at given location and time",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": { "type": "string", "description": "Latitude in decimal fomamat. Eg: 17.3851. Range (-90; 90)"},
                    "longitude": { "type": "string", "description": "Longitude in decimal fomamat. Eg: 78.48. (-180; 180)"},
                    "time": { "type": "string", "description": "Timestamp (Unix time, UTC time zone), e.g. dt=1586468027. Data is available from January 1st, 1979 till 4 days ahead" }
                },
                "required": ["latitude", "longitude", "time"],
                "additionalProperties": False,
            },
            "strict": True,
        }

weather_api_key = userdata.get('WEATHER_API')
x_rapidapi_host = userdata.get('x-rapidapi-host')

import http.client

def get_weather_at(latitude, longitude, time):
  conn = http.client.HTTPSConnection("open-weather13.p.rapidapi.com")
  headers = { 'x-rapidapi-key': weather_api_key,
              'x-rapidapi-host': x_rapidapi_host }
  conn.request("GET", f"/fivedaysforcast?latitude={latitude}&longitude={longitude}&lang=EN", headers=headers)
  res = conn.getresponse()
  return res.read()


response = client.responses.create(
    model="gpt-4o-mini",
    input="What is the weather in hyderabad now",
    tools=[weather_tool],
)

# print("═══ Step 2: Model returns a function_call (not text!) ═══")
# for item in response.output:
#     print(f"  {item.type}: ", end="")
#     if item.type == "function_call":
#         print(f"{item.name}({item.arguments})")
#     else:
#         print(getattr(item, 'content', ''))

# Step 3: We execute the function with the model's arguments
function_call = [item for item in response.output if item.type == "function_call"][0]
args = json.loads(function_call.arguments)

# result = add(**args)       # add(a=7, b=12) → 19 # Commented out, 'add' function is not defined
# print(f"═══ Step 3: We ran add({args}) → {result} ═══")

# To actually call the function extracted by the model:
if function_call.name == "get_weather_at":
    # Note: args will contain string values for latitude, longitude, and time.
    # You might need to convert them to appropriate types if your function expects them.
    # For this specific function, they are already strings, which is fine.
    print(f"Calling {function_call.name} with arguments: {args}")
    result = get_weather_at(**args)
    print(f"═══ Step 3: We ran add({args}) → {result} ═══")

Calling get_weather_at with arguments: {'latitude': '17.3851', 'longitude': '78.4867', 'time': '1701253547'}
═══ Step 3: We ran add({'latitude': '17.3851', 'longitude': '78.4867', 'time': '1701253547'}) → b'{"cod":"200","message":0,"cnt":40,"list":[{"dt":1772290800,"main":{"temp":302.38,"feels_like":301.9,"temp_min":301.5,"temp_max":302.38,"pressure":1008,"sea_level":1008,"grnd_level":946,"humidity":39,"temp_kf":0.88},"weather":[{"id":802,"main":"Clouds","description":"scattered clouds","icon":"03n"}],"clouds":{"all":40},"wind":{"speed":3.74,"deg":118,"gust":6.02},"visibility":10000,"pop":0,"sys":{"pod":"n"},"dt_txt":"2026-02-28 15:00:00"},{"dt":1772301600,"main":{"temp":301.7,"feels_like":301.09,"temp_min":300.35,"temp_max":301.7,"pressure":1008,"sea_level":1008,"grnd_level":946,"humidity":37,"temp_kf":1.35},"weather":[{"id":802,"main":"Clouds","description":"scattered clouds","icon":"03n"}],"clouds":{"all":28},"wind":{"speed":2.42,"deg":167,"gust":3.5},"visibility":10000,"pop":0,"sys

In [25]:
# Step 4: Send the result back to the model
# We need to include:
#   - The model's original output (preserves its chain of thought)
#   - Our function_call_output (the actual result, linked by call_id)

#followup_input = list(response.output)     # the model's original items
followup_input = []
followup_input.append({
    "type": "function_call_output",        # this is OUR result going back
    "call_id": function_call.call_id,      # links to the specific function_call
    "output": str(result),                 # must be a string
})

# Step 5: Model writes the final answer using the real result
final = client.responses.create(
    model="gpt-4o-mini",
    instructions="Use weather_tool to get weather data",
	previous_response_id=response.id,   # links to the previous response
    input=followup_input,
    tools=[weather_tool],
)

print(f"═══ Step 5: Final answer ═══")
print(final.output_text)

═══ Step 5: Final answer ═══
The current weather in Hyderabad is as follows:

- **Temperature:** 29.2°C
- **Feels Like:** 29.1°C
- **Weather Condition:** Scattered clouds
- **Humidity:** 39%
- **Wind Speed:** 3.74 m/s, coming from the southeast
- **Visibility:** 10 km

If you need more specific information or forecasts, feel free to ask!


#